# 0 - Before starting

In [ ]:
pip install tcp-sdk

TCP SDK allows you to :
- manage safe connections to the API
- easily send http requests to the API thanks to the method query()

## a. Requirements

In [ ]:
import os
from tcp import client
import tcp
import pprint
pp=pprint.PrettyPrinter(indent=4)

If you are using Google Collab please execute the cell below as well 

In [ ]:
#from google.colab import files

## b. Connection informations

In [ ]:
email=""
password = ""

# 1 - Authentication

## a. User Login

User login is done by client object instanciation:

In [ ]:
api=client(usermail=email,passwd=password)

You can now retrieve your token as follows:

In [ ]:
token=api.token
print(token)

TIPS: Once you retrieved your token, you can add the token to the env variable "TCP_API_TOKEN". 
After that, you won't need to provide email and password to create a connection.

A token is only valid for a given period so you will need to reconnect from time to time

### Request structure with TCP-SDK:

Now that you are authenticated, request are generated threw the query method as follow: 

api.query().endpoint1.endpoint2.httpMethod(body_data,url_parameters)

ex:

-api.query().auth.get()

-api.query().app.run.post({"inputs":{"pcs":clouds,"trj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"]},Domain="railway",App="rail-modeling")

### TIPS - Finding endpoints - help method

TCP sdk provides a help method which list all endpoints available and specific functions.

It can be called:

#### - on the whole module:

In [ ]:
api.help()

#### - or on any endpoints

In [ ]:
api.query().app.help()

#### - another endpoint

In [ ]:
api.query().app.run.help()

____________________________________________________________________________________________________
Now, let's go back to the "auth" section:

## b. Get user's informations

This endpoints provide user informations such as id, mail, creation date, associated groups...

In [ ]:
#Method=GET
#Endpoints=auth/

user_info=api.query().auth.get()
pp.pprint(user_info)

## c. Get token's informations

This endpoint verifies the validity of your token and provides some information about it, including token id, user id, expiration date, eventual activation limits, number of uses, scope authorized ...

In [ ]:
#Method=GET
#Endpoints=auth/validate_auth_token/

pp.pprint(api.query().auth.validate_auth_token.get())


## d. User logout

In [ ]:
#Method=GET
#Endpoints=auth/logout

api.query().auth.logout.get()

# 2 - Manage your licences

## a. Get licence information

This endpoints provides a list of licenses associated with your account.

In [ ]:
#Method=GET
#Endpoints=lics/

pp.pprint(api.query().lics.get())

# 3 - Manage your data

This section, allows you to check your data, upload new data from your local repository to the S3 storage or, on the contrary, download data from S3 storage to your local repository.

In order to make it user friendly, we created the functions upload() and download(). 

Path in S3 repository are formulated from your personnal folder. It means that:
path=file.txt points towards a file directly in your personnal folder
path=folder/file.txt points towards a file in a folder in your personnal space.

Addressing groups space is done through @:

ex: GroupName@folder/file.txt

Spaces and special characters are not allowed.


## a. List your TCP data

In [ ]:
#Method=GET
#Endpoints=data/

pp.pprint(api.query().data.get())

## b. Upload Data

##### Local file path definition:

If you are using Google collab, please execute the cell below. It will upload your file from your local path to collab's drive

In [ ]:
#uploaded = files.upload()
#file_name=next(iter(uploaded))
#local_file_path=os.path.join("/content", file_name)

If you are using Jupyter, please provide the path of your file below:

In [ ]:
local_file_path=r""

##### Upload in your s3 repository:

In [ ]:
#Definition of target location in S3 repository
s3_target_uri="test_upload"

api.upload(local_file_path,s3_target_uri)

##### Upload in your s3 repository, in a folder:

In [ ]:
#Definition of target location in S3 repository
s3_target_uri="folder/test_upload"

api.upload(local_file_path,s3_target_uri)

##### Upload in your s3 repository, in a group:

In [ ]:
#Definition of target location in S3 repository
s3_target_uri="DB_Engineering@folder/test_upload"

api.upload(local_file_path,s3_target_uri)

## c. Download data

In [ ]:
s3_file_path="test_upload"

# path in your computer. 
# For google collab, the path is the Drive path.The file will be downloaded in your "Download" folder
target_local_path=r""

In [ ]:
api.download(s3_file_path,target_local_path)

If you are using google collab, the precedent command line downloaded the file in collab's drive repository. 
You need to execute the cell below to download the file from the drive to your machine

In [ ]:
#files.download(target_local_path)

## d. Delete files

In [ ]:
file_path_to_delete="test_upload"
api.query().data.delete_file.post({"uri":file_path_to_delete})

# 4 - TCP solutions

## a. List TCP Solutions

This endpoints provides a list of products that are available to your account, sorted by domains.

In [ ]:
#Method=GET
#Endpoints=app/

print(api.query().app.get())


## b. Specific informations for Domain

This endpoint provides a detailed documentation on how to use the solution for a given application. It lists all the available endpoints and their corresponding functionalities. Additionally, it provides detailed information on the required JSON body for each endpoint. The fields in the JSON body are described with their name, type, and a brief description of their purpose. This documentation serves as a comprehensive guide for you to effectively use the solution.

In [ ]:
#Method=GET
#Endpoints=app/info
#Url argument= Domain="railway"& App="rail-modeling"

print(api.query().app.info.get(Domain="railway",App="rail-modeling").decode('utf8'))

## c. Run TCP application

This endpoint is used to initiate a process, where a solution is executed with a given set of parameters.

At least two parameters are required: "inputs" and "output-prefix". Those parameters must strictly adhere to the documentation provided by the endpoint GET /v1/app/info.

Additionally, you can use the parameter "pool" to select a specific architecture on which to run the process. This parameter is a list of instances, that you have selected. The most suitable will be selected in this list. Refer to the endpoint GET /v1/app/remote. If the parameter "pool" is omitted, the run will be run on a small architecture by default.

Upon successful initiation, an ID is returned to you. This ID will be used in subsequent requests to check the status of the process and retrieve the results.

In [ ]:
#inputs definition
clouds=["DB_Engineering@demo/cloud1.laz"]
trj=["DB_Engineering@demo/voie1.dxf"]
output="demo/02_Vecto"

In [ ]:
#Method=POST
#Endpoints=app/info
#Url argument= Domain="railway"& App="rail-modeling"
#data body= {"inputs":{"pcs":clouds,"traj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"]}

resp=api.query().app.run.post({"inputs":{"pcs":clouds,"trj":trj},"output-prefix": output, "pool":["scw:fr-par-1:PRO2-M"]},Domain="railway",App="rail-modeling")
print(resp)

## d. Check process status & download results

Use this endpoint to check the status of a process and retrieve its outputs once it has finished. The following fields are returned: process id, user id, application used, domain, dates of launch and termination, state of the process.

The "outputs" field in the response is a dictionary that contains the files generated by the process. The keys in the dictionary represent the names of the files, and the corresponding values are URLs for download.

The "outputs" field in the response is a dictionary that contains the files generated by the process. The keys in the dictionary represent the names of the files, and the corresponding values are URLs for download.

In [ ]:
#Method=GET
#Endpoints=app/status

id_process = resp['id']

pp.pprint(api.query().app.status.get(Process=id_process))


## e. List Processes

Use this endpoint to retrieve information about all the processes you currently have, including their status and output.

In [ ]:
#Method=GET
#Endpoints=app/list/Process

pp.pprint(api.query().app.list.Process.get())

Alternatively but not in this notebook, you could use, in a terminal, the method api.dashboard() to see a dynamic display 
of the current processes

## f. List Available remotes

The following endpoint provides a comprehensive list of all available Scaleway computing resources, including information such as the number of cores, size of random access memory (RAM), and hourly cost. In addition, if you have declared remotes, they will also appear in this list.

In [ ]:
#Method=GET
#Endpoints=app/remote/scw

pp.pprint(api.query().app.remote.scw.get())